In [1]:
import torch
import torch.nn as nn
import numpy as np

In [2]:
text = """How that personage haunted my dreams, I need scarcely tell you. On
stormy nights, when the wind shook the four corners of the house and
the surf roared along the cove and up the cliffs, I would see him in a
thousand forms, and with a thousand diabolical expressions. Now the leg
would be cut off at the knee, now at the hip, now he was a monstrous
kind of a creature who had never had but the one leg, and that in the
middle of his body. To see him leap and run and pursue me over hedge and
ditch was the worst of nightmares. And altogether I paid pretty dear for
my monthly fourpenny piece, in the shape of these abominable fancies"""

text = text.replace(',','').replace('.','').lower().split()

In [3]:
corpus = set(text)
corpus_length = len(corpus)

word_dict = {}
inverse_word_dict = {}

for i, word in enumerate(corpus):
    word_dict[word] = i
    inverse_word_dict[i] = word

data = []

for i in range(2, len(text) - 2):
    sentence = [text[i-2], text[i-1],
               text[i+1], text[i+2]]
    target = text[i]
    data.append((sentence, target))
    
print(data[3])

(['haunted', 'my', 'i', 'need'], 'dreams')


In [4]:
embedding_length = 20

class CBoW(torch.nn.Module):

    def __init__(self, corpus_length, embedding_dim):
        super(CBoW, self).__init__()
        
        self.embeddings = nn.Embedding(corpus_length, embedding_dim)

        self.linear1 = nn.Linear(embedding_dim, 64)
        self.linear2 = nn.Linear(64, corpus_length)
        
        self.activation_function1 = nn.ReLU()
        self.activation_function2 = nn.LogSoftmax(dim = -1)

    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs)).view(1,-1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out

    def get_word_emdedding(self, word):
        word = torch.LongTensor([word_dict[word]])
        return self.embeddings(word).view(1,-1)

In [5]:
model = CBoW(corpus_length, embedding_length)

loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

def make_sentence_vector(sentence, word_dict):
    idxs = [word_dict[w] for w in sentence]
    return torch.tensor(idxs, dtype=torch.long)

print(make_sentence_vector(['stormy','nights','when','the'], word_dict))

tensor([ 9, 48, 43, 70])


In [6]:
for epoch in range(100):
    epoch_loss = 0
    for sentence, target in data:
        model.zero_grad()
        sentence_vector = make_sentence_vector(sentence, word_dict)  
        log_probs = model(sentence_vector)
        loss = loss_function(log_probs, torch.tensor([word_dict[target]], dtype=torch.long))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.data
    print('Epoch: '+str(epoch)+', Loss: ' + str(epoch_loss.item()))

Epoch: 0, Loss: 540.2003173828125
Epoch: 1, Loss: 479.9447937011719
Epoch: 2, Loss: 435.550537109375
Epoch: 3, Loss: 396.0237731933594
Epoch: 4, Loss: 357.2877502441406
Epoch: 5, Loss: 318.30047607421875
Epoch: 6, Loss: 279.1165771484375
Epoch: 7, Loss: 240.94815063476562
Epoch: 8, Loss: 204.82603454589844
Epoch: 9, Loss: 171.59759521484375
Epoch: 10, Loss: 142.0783233642578
Epoch: 11, Loss: 116.167236328125
Epoch: 12, Loss: 94.1119384765625
Epoch: 13, Loss: 75.69329833984375
Epoch: 14, Loss: 60.69060516357422
Epoch: 15, Loss: 48.82771682739258
Epoch: 16, Loss: 39.65796661376953
Epoch: 17, Loss: 32.629825592041016
Epoch: 18, Loss: 27.246349334716797
Epoch: 19, Loss: 23.11562728881836
Epoch: 20, Loss: 19.909191131591797
Epoch: 21, Loss: 17.365760803222656
Epoch: 22, Loss: 15.346717834472656
Epoch: 23, Loss: 13.695038795471191
Epoch: 24, Loss: 12.3384370803833
Epoch: 25, Loss: 11.209527969360352
Epoch: 26, Loss: 10.247206687927246
Epoch: 27, Loss: 9.425829887390137
Epoch: 28, Loss: 8.721

In [7]:
def get_predicted_result(input, inverse_word_dict):
    index = np.argmax(input)
    return inverse_word_dict[index]

def predict_sentence(sentence):
    sentence_split = sentence.replace('.','').lower().split()
    sentence_vector = make_sentence_vector(sentence_split, word_dict)
    prediction_array = model(sentence_vector).data.numpy()
    print('Preceding Words: {}\n'.format(sentence_split[:2]))
    print('Predicted Word: {}\n'.format(get_predicted_result(prediction_array[0], inverse_word_dict)))
    print('Following Words: {}\n'.format(sentence_split[2:]))

predict_sentence('to see leap and')

Preceding Words: ['to', 'see']

Predicted Word: him

Following Words: ['leap', 'and']



In [9]:
print(model.get_word_emdedding('leap'))

tensor([[ 8.6251e-01, -3.6810e-01, -1.1397e+00, -4.2561e-02,  3.1203e-01,
         -6.3440e-01,  1.0768e+00,  2.7745e-01, -5.6835e-01, -2.0671e+00,
          2.3117e-01, -5.5059e-02,  1.8441e+00, -1.7148e-01,  8.4483e-01,
         -6.5940e-02,  1.2200e+00,  1.0388e+00, -3.9075e-04,  1.3893e+00]],
       grad_fn=<ViewBackward>)
